In [1]:
from datetime import datetime  # <1>
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pathlib as pl
import torch
import torch.utils.data as D
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np

train_losses=[]
valid_losses=[]

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1792, 1024, kernel_size=3, padding=1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features=1024)
        self.conv2 = nn.Conv2d(1024, 512, kernel_size=3, padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features=512)
        self.fc1 = nn.Linear(512 * 4 * 5, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1_batchnorm(self.conv1(x))), 2)
        out = torch.tanh(self.conv2_batchnorm(self.conv2(out)))
        out = out.view(-1, 512 * 4 * 5)
        out = torch.tanh(self.fc1(out))
        out = torch.tanh(self.fc2(out))
        out = self.fc3(out)
        return out

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, valid_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.float().to(device=device).unsqueeze(1)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
        train_losses.append(loss_train/len(train_loader))
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.now(), epoch,
                loss_train / len(train_loader)))
        test(model,valid_loader,device)

def evaluate(model,train_loader,valid_loader,device):
    print(f"Validating on device {device}.")

    model.to(device=device)  # <2>
    model.eval()


    predictions = {}
    for name, loader in [("train", train_loader), ("val", valid_loader)]:
        y_true = torch.tensor([], dtype=torch.float, device=device)
        all_outputs = torch.tensor([], device=device)
        with torch.no_grad():
            for data, labels in loader:
                data = data.to(device=device)
                labels = labels.to(device=device).unsqueeze(1).float()
                y_true = torch.cat((y_true, labels), 0)
                outputs = model(data)
                all_outputs = torch.cat((all_outputs, outputs), 0)
        y_true = y_true.cpu().numpy()
        all_outputs = all_outputs.cpu().numpy()
        predictions[name]=(y_true,all_outputs)
    return predictions

def test(model,valid_loader,device):
    model.eval()

    running_loss = 0.
    correct = 0.
    total = 0.
    loss_fn = nn.MSELoss()

    with torch.no_grad():
        for data,labels in valid_loader:
            data = data.to(device=device)
            labels = labels.to(device=device).unsqueeze(1).float()
            outputs = model(data)

            loss = loss_fn(outputs, labels)
            running_loss += loss.item()

            total += labels.size(0)
            # correct += torch.round(outputs).eq(labels).sum().item()

    valid_loss = running_loss / len(valid_loader)
    # accu = 100. * correct / total

    valid_losses.append(valid_loss)

train_path = pl.Path('D:/results/Feature extraction/flir dataset/fm_kaist_density_10_8/kaist_results_train.pt')
valid_path = pl.Path('D:/results/Feature extraction/flir dataset/fm_kaist_density_10_8/kaist_results_valid.pt')

dateTimeObj = datetime.now()

outdir = pl.Path.joinpath(train_path.parent,str(dateTimeObj.year)+ '_' + \
                          str(dateTimeObj.month) + '_' + \
                          str(dateTimeObj.day) + '_' + str(dateTimeObj.hour) + '_' + \
                          str(dateTimeObj.minute) + '_' + str(dateTimeObj.second))


device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

# train loader
train_label_path = pl.Path.joinpath(train_path.parent, train_path.stem + '_labels' + train_path.suffix)
t1 = torch.load(train_path)
t1 = t1.reshape(8862, 1792, 8, 10)
t2 = torch.load(train_label_path).reshape(8862)
print("Loaded train features from {},labels from {}".format(str(train_path),str(train_label_path)))
trainset = list(zip(t1, t2))
train_loader = D.DataLoader(trainset, batch_size=64,
                            shuffle=True)

# test loader
valid_label_path = pl.Path.joinpath(valid_path.parent, valid_path.stem + '_labels' + valid_path.suffix)
v1 = torch.load(valid_path)
v1 = v1.reshape(1366, 1792, 8, 10)
v2 = torch.load(valid_label_path).reshape(1366)
print(f"Loaded valid features from {str(valid_path)},labels from {str(valid_label_path)}")
valset = list(zip(v1, v2))
valid_loader = torch.utils.data.DataLoader(valset, batch_size=64,
                                          shuffle=False)


model = Net().to(device=device)  # <1>
optimizer = optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()  #  <4>

training_loop(  # <5>
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    valid_loader=valid_loader
)

predictions = evaluate(model,train_loader,valid_loader,device)

train_predict = np.array(predictions['train'])
train_predict = train_predict.reshape(2,train_predict.shape[1]).transpose()
eval_predict = np.array(predictions['val'])
eval_predict = eval_predict.reshape(2,eval_predict.shape[1]).transpose()

print(f"Train MSE: {mean_squared_error(train_predict[:,0],train_predict[:,1]):.2f}")
print(f"Valid MSE: {mean_squared_error(eval_predict[:,0],eval_predict[:,1]):.2f}")

if not outdir.exists():
    outdir.mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), pl.Path.joinpath(outdir, 'trained_model.pt'))

np.save(pl.Path.joinpath(outdir,'mse_train_arr'),train_predict)
np.save(pl.Path.joinpath(outdir,'mse_valid_arr'),eval_predict)

mse_train_by_num= [mean_squared_error(train_predict[train_predict[:, 0] == i][:, 0] \
                                            ,train_predict[train_predict[:, 0] == i][:, 1]) \
                          for i in np.unique(train_predict[:,0])]

mse_val_by_num = [mean_squared_error(eval_predict[eval_predict[:, 0] == i][:, 0] \
                                        , eval_predict[eval_predict[:, 0] == i][:, 1]) \
                    for i in np.unique(eval_predict[:, 0])]

fig = plt.figure()
plt.plot(train_losses)
plt.plot(valid_losses)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('losses')
plt.legend(['Train', 'Valid'])
plt.title('Train vs Valid Losses')
plt.savefig(pl.Path.joinpath(outdir,'train_loss.png'))

fig = plt.figure()
plt.plot(mse_train_by_num)
plt.plot(mse_val_by_num)
plt.grid()
plt.xlabel('person count')
plt.ylabel('mse')
plt.legend(['Train', 'Valid'])
plt.title('Train vs Valid MSE by number of people')
plt.savefig(pl.Path.joinpath(outdir,'train_mse_by_people.png'))
# plt.show()